# CNN model

In [1]:
# Imports

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Local test setup
tf.config.threading.set_inter_op_parallelism_threads(6) # 8
tf.config.threading.set_intra_op_parallelism_threads(20) # 24

### 1. Data Preprocessing

In [ ]:
# Loading CIFAR-10 dataset

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('Training set shape:', x_train.shape)
print('Testing set shape:', x_test.shape)

print('Training labels shape:', y_train.shape)
print('Testing labels shape:', y_test.shape)

labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

print(labels)

In [ ]:
# Visualizing some images

plt.figure(figsize = (8, 8))

for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(x_train[i])
    plt.xlabel(labels[y_train[i][0]])

plt.show()

In [4]:
# One-hot encoding the labels

y_train = tf.keras.utils.to_categorical(y_train, num_classes = 10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes = 10)

In [ ]:
# Normalizing the Images

x_train = x_train / 255
x_test = x_test / 255

print(x_train[0].min())
print(x_train[0].max())

In [ ]:
# Resizing images

x_train = tf.image.resize(x_train, (64, 64))
x_test = tf.image.resize(x_test, (64, 64))

print('Resized training image size:', x_train[0].shape)
print('Resized testing image size:', x_test[0].shape)

In [7]:
# Creating validation set

x_train = np.array(x_train)
y_train = np.array(y_train)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.1, random_state = 42)

# Augmenting data

augment = ImageDataGenerator(
    horizontal_flip = True,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    # rotation_range = 10,      # Most images in the dataset have a natural, upright orientation.
    zoom_range = 0.1)

augment.fit(x_train)

### 2. Model Architecture

In [ ]:
# Defining the model

model = Sequential()

model.add(Conv2D(64, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform', kernel_regularizer = l2(0.01), input_shape = (64, 64, 3)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform', kernel_regularizer = l2(0.01)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform', kernel_regularizer = l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform', kernel_regularizer = l2(0.01)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform', kernel_regularizer = l2(0.01)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform', kernel_regularizer = l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform', kernel_regularizer = l2(0.01)))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform', kernel_regularizer = l2(0.01)))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform', kernel_regularizer = l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(256, activation = 'relu', kernel_initializer = 'he_uniform', kernel_regularizer = l2(0.01)))
model.add(Dropout(0.3))

model.add(Dense(128, activation = 'relu', kernel_initializer = 'he_uniform', kernel_regularizer = l2(0.01)))
model.add(Dropout(0.3))

model.add(Dense(10, activation = 'softmax'))

# Model summary

model.summary()

### 3. Model Training

In [ ]:
# Compiling the model

# Early stopping / Learning rate scheduler
stopping = EarlyStopping(monitor = 'val_loss', patience = 10, restore_best_weights = True)
scheduler = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 3, min_lr = 1e-6)

model = load_model('../models/senimtra_cnn_90.keras')

model.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])

# Training the model (done in Colab)

# model.fit(augment.flow(x_train, y_train, batch_size = 256), epochs = 50, validation_data = (x_val, y_val), callbacks = [scheduler, stopping])

# Evaluating the model on the test data

test_loss, test_accuracy = model.evaluate(x_test, y_test)

print('Test data loss:', test_loss)
print('Test data accuracy:', test_accuracy)

### 4. Model Evaluation

In [ ]:
# Evaluating the model on the validation set

val_loss, val_accuracy = model.evaluate(x_val, y_val)

print('Validation data loss:', val_loss)
print('Validation data accuracy:', val_accuracy)

In [ ]:
# Computing accuracy, precision, recall and F1-score

y_pred = model.predict(x_val)
y_pred_labels = np.argmax(y_pred, axis = 1)
y_true = np.argmax(y_val, axis = 1)

report = classification_report(y_true, y_pred_labels, target_names = labels)

print(report)

In [ ]:
# Visualizing confusion matrix

cm = confusion_matrix(y_true, y_pred_labels)

plt.figure(figsize = (8, 6))
sns.heatmap(cm, annot = True, fmt = 'd', cmap = 'Blues', xticklabels = labels, yticklabels = labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

### 5. Shrink Model Size

In [ ]:
def convert_to_float16(model):
    float16_model = tf.keras.models.clone_model(model)
    for layer in float16_model.layers:
        layer.set_weights([w.astype('float16') for w in model.get_layer(name = layer.name).get_weights()])
    return float16_model

float16_model = convert_to_float16(model)

float16_model.save('../models/senimtra_cnn_90.keras')

# Evaluating the resized model

float16_model.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])

val_loss, val_accuracy = float16_model.evaluate(x_val, y_val)

print('Validation data loss:', val_loss)
print('Validation data accuracy:', val_accuracy)